# Table S4. Structural features of pri-miRNAs
1. Generate all structures
2. Select one structure
3. Draw figures
4. Determine junctions
5. Get features
6. Build a table

In [1]:
import time
today = time.strftime('%Y-%m-%d')
name = 'S. Chan Baek'
print 'Last revised by %s at %s.' % (name, today)

Last revised by S. Chan Baek at 2024-01-19.


In [2]:
HOME = '/casa/bsc/projects/2_Structure-of-pri/2007_paper_prep'
%cd $HOME

/casa/bsc/projects/2_Structure-of-pri/2007_paper_prep


In [4]:
from __future__ import division
from collections import Counter, defaultdict
from os import path, listdir
import pandas as pd
import numpy as np
import re
import gzip

In [5]:
LIBRARIES = [ 'rep1', 'rep2-1', 'rep2-2', 'rep2-3' ]
CONDITIONS = [ '+1m7', '-1m7', 'dc' ]

In [101]:
s1 = pd.read_csv('publication/TableS1__Pri-miRNAs_selected_for_SHAPE-MaP.csv',index_col=0)
s2 = pd.read_csv('publication/TableS2__Constructs_used_in_this_study.csv',index_col=0)
s3 = pd.read_csv('publication/TableS3_2__SHAPE_reactivity.csv',index_col=0)
s3cnts = pd.read_csv('publication/TableS3_1__Read_counts.csv',index_col=0)
s3shan = pd.read_csv('publication/TableS3_3__Shannon_entropy.csv',index_col=0)
s3shan.columns = range(1,126)
s3shan = s3shan.fillna(0)
constseqs = s2['Construct sequence'].to_dict()
mirs = [m for m in s1.index if m in s3.index]
print len(s1.index), len(mirs)

519 478


In [7]:
def get_pre_position(mir):
    constseq = constseqs[mir].replace('T','U')
    preseq = s1.loc[mir, 'Precursor']
    return constseq.find(preseq)+1, constseq.find(preseq)+len(preseq)

def get_mat_position(mir,arm):
    constseq = constseqs[mir].replace('T','U')
    matseq = s1.loc[mir,arm]
    return constseq.find(matseq)+1, constseq.find(matseq)+len(matseq)

m = 'hsa-let-7a-1'
print get_pre_position(m), get_mat_position(m,'5p')

(25, 96) (25, 46)


In [8]:
fold = '/casa/bsc/bin/RNAstructure/exe/Fold'
datapath = '/casa/bsc/bin/RNAstructure/data_tables'
pvclient = '/casa/bsc/bin/ShapeMapper_v1.2/pvclient.py'
%env DATAPATH $datapath

env: DATAPATH=/casa/bsc/bin/RNAstructure/data_tables


---
### 1. Generate all possible structure models
- shape reactivity files were generated in "TableS3__SHAPE_reactivity"

In [12]:
RCUT_PD = 500
RCUT_M = 100
sufflibs = {}
for m in mirs:
    sufflibs[m] = [lib for lib in LIBRARIES if s3cnts.loc[m,'%s_+1m7'%lib]>=RCUT_PD and 
                                               s3cnts.loc[m,'%s_-1m7'%lib]>=RCUT_M and
                                               s3cnts.loc[m,'%s_dc'%lib]>=RCUT_PD ]

In [18]:
PARAMS = [ (1.8,-.6), (2.4,-.8), (1.2,-.4) ]
FASTA_DIR = 'resources/fasta_files'
SHAPE_DIR = 'shape/2021-12-28'
RTS_DIR = '%s/shape_reactivity'%SHAPE_DIR
STR_DIR = '%s/structures'%SHAPE_DIR
FIG_DIR = '%s/figures'%SHAPE_DIR

In [13]:
# for rna in s2.index:
#     with open('%s/%s.fa'%(FASTA_DIR,rna),'wt') as out:
#         out.write('>%s\n%s'%(rna,constseqs[rna]))

In [14]:
# ## Generate ct files
# if not path.exists(STR_DIR): 
#     !mkdir $STR_DIR
    
# for m in mirs:
#     seqfile = '%s/%s.fa'%(FASTA_DIR,m)
#     for lib in sufflibs[m]:
#         rtsfile = '%s/%s_%s.shape'%(RTS_DIR,m,lib)
#         for slope,inter in PARAMS:
#             ctfile = '%s/%s_%s_m%i.ct'%(STR_DIR,m,lib,slope*10)
#             out = !$fold $seqfile $ctfile -sh $rtsfile -sm $slope -si $inter

---
### 2. Select the most relevant structure model
- First, select those with some lengths of upper and lower stems, 2 nt overhang, and a symmetric stem.
- Second, give priority to those modeled with more reads.
- For leftovers, loosening the threshold step by step.
- If there are still more than one, select those correponds best with SHAPE reactivity profile.

In [20]:
def split_ct(ctfile):
    infos = open(ctfile, 'rt').read()
    rna = infos.split('\n')[0].strip().split()[-1]
    nstrt = infos.count(rna)
    splitinfos = []
    for i in range(nstrt):
        lines = infos.split('\n')[1+(1+len(constseqs[rna]))*i:(1+len(constseqs[rna]))*(i+1)]
        splitinfos.append([ tuple(map(int,l.strip().split()[4:6][::-1])) for l in lines ])
    return splitinfos

def count_ustempairs(ctinfo, ps, pe, me5p, ms3p):
    return len([(i,p) for i,p in ctinfo if ps<=i<=me5p and ms3p-2<=p<=pe-2])

def count_lstempairs(ctinfo, ps, pe):
    return len([(i,p) for i,p in ctinfo if ps-13<=i<ps and pe-2<p<=pe+11])

def measure_asymmetry(ctinfo, ps, pe, me5p, ms3p):
    pairs = [(i,p) for i,p in ctinfo if i<me5p and p>=ms3p]
    stems = [(i,p) for i,p in pairs if ps-9<=i<=ps+15 or pe-15<=p<=pe+8]
    consecps = zip(stems[:-1],stems[1:])
    return max([abs((i2-i1)-(p1-p2)) for (i1,p1),(i2,p2) in consecps]+[-999])

def get_sym_str(ctinfo, ps, pe):
    stinfo = [(x,y) for x,y in ctinfo if y>0]
    ls = [ (x,y) for i,(x,y) in enumerate(stinfo[:-1]) 
          if stinfo[i+1]==(y,x) and x>ps and y<pe ]
    loopst, loopen = min(ls, key=lambda (x,y):abs(120-x-y))
    stems = [ (i,p) for i,p in ctinfo if i<=loopst and loopen<=p ]
    if len(stems)<2:
        return 'F'*125
    stemst, stemen = stems[0]
    stems = stems + [ (p,i) for i,p in stems[::-1] ]
    symst = ''
    for (l5,l3),(u5,u3) in zip(stems[:-1],stems[1:]):
        if l5==loopst: symst += 'M'+'L'*(loopen-loopst-1)
        elif u5-l5==1: symst += 'M'
        else:
            symmet = min(u5-l5-1, l3-u3-1)
            symst += 'M' + symmet*'S' + (u5-l5-1-symmet)*'A'# + symmet*'S'
    symst = 'F'*(stemst-1) + symst + 'M' + 'F'*(125-stemen)
    return symst

def count_len(symstr):
    return sum([symstr.count(s) for s in 'MS'])

def measure_3overhang(ctinfo, ps, pe):
    termps = [(i,j) for i,j in ctinfo if 0<=i-ps<=5 and 2<=pe-j<=7]
    intst = [(i,j) for i,j in ctinfo if ps-10<i<j<ps+10 or pe-10<i<j<pe+10]
    if termps==[] or intst:
        return -999
    symstr = get_sym_str(ctinfo, ps, pe)
    ov3 = count_len(symstr[:ps-1])-count_len(symstr[pe:]) # (pe-lastp[1])-(lastp[0]-ps)
    return ov3

In [21]:
mir = 'hsa-mir-127'
ctfile = '%s/%s_rep1_m18.ct'%(STR_DIR,mir)
ctinfo = split_ct(ctfile)[1]
ms5p, me5p = get_mat_position(mir,'5p')
ms3p, me3p = get_mat_position(mir,'3p')
print count_ustempairs(ctinfo, ms5p, me3p, me5p, ms3p)
print count_lstempairs(ctinfo, ms5p, me3p)
print measure_3overhang(ctinfo, ms5p, me3p)
print measure_asymmetry(ctinfo, ms5p, me3p, me5p, ms3p)

19
8
2
3


In [49]:
PARAMS = [18,12,24]
MINUS = 14
MINLS = 8
OVRANGE = range(1,4)
MAXASYM = 8

In [50]:
strmodels = defaultdict(list)
for m in mirs:
    ms5p, me5p = get_mat_position(m,'5p')
    ms3p, me3p = get_mat_position(m,'3p')
    for lib in sufflibs[m]:
        for param in PARAMS:
            ctfile = '%s/%s_%s_m%s.ct'%(STR_DIR,m,lib,param)
            for i,ctinfo in enumerate(split_ct(ctfile)):
                usbps = count_ustempairs(ctinfo, ms5p, me3p, me5p, ms3p)
                ov3 = measure_3overhang(ctinfo, ms5p, me3p)
                lsbps = count_lstempairs(ctinfo, ms5p, me3p)
                asym = measure_asymmetry(ctinfo, ms5p, me3p, me5p, ms3p)
                if usbps>=MINUS and ov3 in OVRANGE and lsbps>=MINLS and asym<=MAXASYM:
                    strmodels[m].append((ctinfo,ctfile,i+1,lib))
print len(strmodels)

428


In [51]:
RCUT = 5000
cnt = 0
for m in strmodels:
    priorlibs = [lib for lib in sufflibs[m] if s3cnts.loc[m,['%s_+1m7'%lib,'%s_-1m7'%lib,'%s_dc'%lib]].mean()>=RCUT]
    priorstrs = [st for st in strmodels[m] if st[3] in priorlibs]
    if priorlibs and priorstrs:
        strmodels[m] = priorstrs
        cnt += 1
print cnt

373


In [53]:
ovrange = OVRANGE
minls = MINLS
maxasym = MAXASYM
REL_ASYM = 20
REL_OVER = range(-4,6)

while len(strmodels)<len(mirs) and minls>=0:
    leftover = [m for m in mirs if m not in strmodels]
    if ovrange==OVRANGE and minls==1:
        ovrange = REL_OVER
        minls = MINLS
        maxasym = MAXASYM
    elif maxasym==REL_ASYM:
        minls -= 1
    else:
        maxasym += 2
        
    for m in leftover:
        ms5p, me5p = get_mat_position(m,'5p')
        ms3p, me3p = get_mat_position(m,'3p')
        for lib in sufflibs[m]:
            for param in PARAMS:
                ctfile = '%s/%s_%s_m%s.ct'%(STR_DIR,m,lib,param)
                for i,ctinfo in enumerate(split_ct(ctfile)):
                    usbps = count_ustempairs(ctinfo, ms5p, me3p, me5p, ms3p)
                    ov3 = measure_3overhang(ctinfo, ms5p, me3p)
                    lsbps = count_lstempairs(ctinfo, ms5p, me3p)
                    asym = measure_asymmetry(ctinfo, ms5p, me3p, me5p, ms3p)
                    if usbps>=MINUS and ov3 in ovrange and lsbps>=minls and asym<=maxasym:
                        strmodels[m].append((ctinfo,ctfile,i+1,lib))
print len(strmodels)

476


In [56]:
from scipy.stats import mannwhitneyu
def get_auc(ctinfo, ctfile):
    m,lib = ctfile.split('_')[:2]; m = m.split('/')[-1]
    rtsfile = '%s/%s_%s.shape'%(RTS_DIR,m,lib)
    rts = !cut -f2 $rtsfile
    rts = map(float,rts)
    rtssrna = [rt for rt,info in zip(rts,zip(*ctinfo)[1]) if info==0]
    rtdsrna = [rt for rt,info in zip(rts,zip(*ctinfo)[1]) if info>0]
    auc = mannwhitneyu(rtssrna,rtdsrna)[0]/len(rtssrna)/len(rtdsrna)
    return max(auc,1-auc)

In [57]:
folded = sorted(strmodels)
ctinfos, ctfileinfos = {}, {}
for m in folded:
    ctinfo,ctfile,i,lib = max(strmodels[m],key=lambda x:get_auc(x[0],x[1]))
    ctinfos[m] = ctinfo
    ctfileinfos[m] = (ctfile,i,lib)

In [58]:
# adjustment
ctfileinfos['hsa-mir-19a'] = ('shape/2021-07-05/structures/hsa-mir-19a_rep2-2_m18.ct',1,'rep2-2')
ctfileinfos['hsa-mir-99a'] = ('shape/2021-07-05/structures/hsa-mir-99a_rep2-1_m18.ct',1,'rep2-1')
ctinfos['hsa-mir-19a'] = split_ct('shape/2021-07-05/structures/hsa-mir-19a_rep2-2_m18.ct')[0]
ctinfos['hsa-mir-99a'] = split_ct('shape/2021-07-05/structures/hsa-mir-99a_rep2-1_m18.ct')[0]

---
### 3. Make the figures of the selected structure models

In [59]:
def split_ct_all(ctfile):
    infos = open(ctfile, 'rt').read()
    rna = infos.split('\n')[0].strip().split()[-1]
    nstrt = infos.count(rna)
    splitinfos = []
    for i in range(nstrt):
        lines = infos.split('\n')[(1+len(constseqs[rna]))*i:(1+len(constseqs[rna]))*(i+1)]
        splitinfos.append(lines)
    return splitinfos

def draw(ct, shape, ctnum, out, mir):
    outf = !$pvclient --ct $ct --shape $shape --out $out --structures $ctnum
    if not mir.startswith('hsa-'):
        return
    
    s5, e5 = get_mat_position(mir,'5p')
    s3, e3 = get_mat_position(mir,'3p')
    boldpos = range(s5,e5+1)+range(s3,e3+1)
    
    eps = '%s.eps'%out
    neweps = eps.replace('.eps','_b.eps')
    newf = open(neweps, 'wt')
    for l in open(eps, 'rt'):
        if l.startswith('% nucleotide'):
            pos = int(l.split()[2]) - 1 # since position starts from 2 in initial file
            if pos in boldpos:
                newf.write('/Helvetica-Bold findfont 12 scalefont setfont\n')
            else:
                newf.write('/Helvetica findfont 10.5 scalefont setfont\n')
        elif l.find('hsa-') >= 0 and l.find('moveto') >= 0:
            rna = l.split()[-2][:-1]
            st = l.find('(') + 1
            en = l.rfind(')')
            l = l.replace(l[st:en], rna)
            newf.write('/Helvetica-Bold findfont 6 scalefont setfont\n')
        newf.write(l.replace('(T)', '(U)'))
    newf.close()
    !rm -f $eps
    return

In [60]:
# if not path.exists(FIG_DIR):
#     !mkdir $FIG_DIR

# ctfiles = {}    
# for m in folded:
#     ctfile, ctnum, lib = ctfileinfos[m]
#     shapefile = '%s/%s_%s.shape'%(RTS_DIR,m,lib)
    
#     select = '%s/%s'%(FIG_DIR,ctfile.split('/')[-1].replace('.ct','_%s.ct'%ctnum))
#     ctlines = split_ct_all(ctfile)[ctnum-1]
#     ctfiles[m] = select
#     with open(select,'wt') as f:
#         f.write('\n'.join(ctlines))
    
#     eps = '%s/%s'%(FIG_DIR,m)
#     if '%s_b.eps'%m in listdir(FIG_DIR):
#         continue
#     draw(ctfile, shapefile, ctnum, eps, m)

---
### 4. Determine the apical and basal junctions

In [61]:
looppos = {}
for m in folded:
    ps, pe = get_pre_position(m)
    stinfo = [ p for p in ctinfos[m] if p[1]>0 ] 
    ls = [ (x,y) for i,(x,y) in enumerate(stinfo[:-1]) 
           if stinfo[i+1]==(y,x) and x>ps and y<pe ]
    if not ls: print m
    looppos[m] = min(ls, key=lambda (x,y):abs(120-x-y))

In [62]:
def get_sym_str(m):
    loopst, loopen = looppos[m]
    stems = [ (i,p) for i,p in ctinfos[m] if i<=loopst and loopen<=p ]
    if len(stems)<2:
        return 'F'*125
    stemst, stemen = stems[0]
    stems = stems + [ (p,i) for i,p in stems[::-1] ]
    symst = ''
    for (l5,l3),(u5,u3) in zip(stems[:-1],stems[1:]):
        if l5==loopst: symst += 'M'+'L'*(loopen-loopst-1)
        elif u5-l5==1: symst += 'M'
        else:
            symmet = min(u5-l5-1, l3-u3-1)
            symst += 'M' + symmet*'S' + (u5-l5-1-symmet)*'A'# + symmet*'S'
    symst = 'F'*(stemst-1) + symst + 'M' + 'F'*(125-stemen)
    return symst

def calculate_dist(pair1, pair2):
    return min(abs(pair1[0]-pair2[0]), abs(pair1[1]-pair2[1]))

def list_jcs(stems): # [ (1, 124), (2, 123), ... ]
    jcpos = []
    for uppair,lowpair in zip(stems[:-1],stems[1:]):
        jcsize = abs(uppair[0]-lowpair[0])+abs(lowpair[1]-uppair[1])-2
        if jcsize>=1:
            jcpos.append(uppair)
    jcpos.append(stems[-1])
    return jcpos

def list_all_junctions(m,option):
    ps, pe = get_pre_position(m)
    loopst, loopen = looppos[m]
    stems = [ (i,p) for i,p in ctinfos[m] if i<=loopst and loopen<=p ]
    stems = [ (i,p) for i,p in stems 
              if not any([i2 for i2,p2 in ctinfos[m] if i<i2<p2<ps or pe<i2<p2<p]) ]
    ajcs = list_jcs(stems)
    bjcs = list_jcs(stems[::-1])
    if option=='basal':
        return bjcs
    return ajcs

def get_delta_g(st5,st3,dg5,dg3):
    if st5==[] or st3==[]:
        return dg5+dg3
    if (st5[0]==0 and st3[0]==0):
        return [dg5[0]+dg3[0]]+get_delta_g(st5[1:],st3[1:],dg5[1:],dg3[1:])
    if st5[0]==0:
        return [dg5[0]]+get_delta_g(st5[1:],st3,dg5[1:],dg3)
    if st3[0]==0:
        return [dg3[0]]+get_delta_g(st5,st3[1:],dg5,dg3[1:])
    return [dg5[0]+dg3[0]]+get_delta_g(st5[1:],st3[1:],dg5[1:],dg3[1:])

In [63]:
print list_all_junctions('hsa-mir-16-1','basal')
print list_all_junctions('hsa-mir-16-1','apical')

[(54, 70), (41, 78), (38, 81), (28, 92), (24, 96), (16, 104)]
[(21, 99), (26, 94), (37, 83), (39, 80), (48, 71), (57, 67)]


In [64]:
WIN = 15
lrange = range(2,1+WIN)
lsets = [(i,j) for i in lrange for j in lrange]
cols = ['mir','dist']+range(-WIN,WIN)

In [65]:
rtstbl = pd.DataFrame(index=sorted(folded), columns=range(1,126))
for m in folded:
    lib = ctfileinfos[m][-1]
    rtsfile = '%s/%s_%s.shape'%(RTS_DIR,m,lib)
    rts = !cut -f2 $rtsfile
    rtstbl.loc[m] = dict(zip(range(1,126),map(float,rts)))
rtstbl.iloc[:3,:10]

,1,2,3,4,5,6,7,8,9,10
hsa-let-7a-1,0.134258,0,0.0821034,0,0.233454,0.440442,0.241292,1.09485,0.0335838,0.260488
hsa-let-7a-2,0,0.184993,0,0.19736,0,0,0.735154,0.242866,0.0719598,0.793133
hsa-let-7a-3,0,0,0,0.133857,0.201411,0.132156,0.0767123,0,0.504929,0.58487


In [66]:
M = 1.8
B = -.6
delgtbl = np.log(rtstbl.astype(float)+1)*M+B

In [68]:
delgbj, delgaj = pd.DataFrame(columns=cols), pd.DataFrame(columns=cols)
for m in folded:
    ps, pe = get_pre_position(m)
    symstr = get_sym_str(m)
    ctinfod = {i:0 if (0<i<60 and 0<p<60) or (60<i and 60<p) else p for i,p in ctinfos[m]}
    for x5,x3 in list_all_junctions(m,'basal'):
        if x5==1 or x3==125:
            continue
        if x5<ps: 
            dist = -count_len(symstr[x5-1:ps-1])
        else:
            dist = count_len(symstr[ps-1:x5])
        
        st5 = [ctinfod[x] if x in ctinfod else 0 for x in range(x5-15,x5)][::-1]
        st3 = [ctinfod[x] if x in ctinfod else 0 for x in range(x3+1,x3+16)]
        dg5 = delgtbl.loc[m,range(x5-15,x5)].copy().fillna(0).tolist()[::-1]
        dg3 = delgtbl.loc[m,range(x3+1,x3+16)].copy().fillna(0).tolist()
        dgs = get_delta_g(st5,st3,dg5,dg3)[:15]
        st5 = [ctinfod[x] if x in ctinfod else 0 for x in range(x5,x5+15)]
        st3 = [ctinfod[x] if x in ctinfod else 0 for x in range(x3-14,x3+1)][::-1]
        dg5 = delgtbl.loc[m,range(x5,x5+15)].copy().fillna(0).tolist()
        dg3 = delgtbl.loc[m,range(x3-14,x3+1)].copy().fillna(0).tolist()[::-1]
        dgs = dgs[::-1]+get_delta_g(st5,st3,dg5,dg3)[:15]
        delgbj = delgbj.append(pd.Series(dict(zip(cols,[m,dist]+dgs))),ignore_index=True)
        
    for x5,x3 in list_all_junctions(m,'apical'):
        if x5<ps: 
            dist = -count_len(symstr[x5-1:ps-1])
        else:
            dist = count_len(symstr[ps-1:x5])
        mid = (x5+x3)//2
        st5 = [ctinfod[x] if x in ctinfod else 0 for x in range(x5-14,x5+1)][::-1]
        st3 = [ctinfod[x] if x in ctinfod else 0 for x in range(x3,x+15)]
        dg5 = delgtbl.loc[m,range(x5-14,x5+1)].copy().fillna(0).tolist()[::-1]
        dg3 = delgtbl.loc[m,range(x3,x3+15)].copy().fillna(0).tolist()
        dgs = get_delta_g(st5,st3,dg5,dg3)[:15]
        st5 = [ctinfod[x] for x in range(x5+1,min(x5+16,mid))]
        st3 = [ctinfod[x] for x in range(max(mid,x3-15),x3)][::-1]
        dg5 = delgtbl.loc[m,range(x5+1,min(x5+16,mid))].copy().fillna(0).tolist()
        dg3 = delgtbl.loc[m,range(max(mid,x3-15),x3)].copy().fillna(0).tolist()[::-1]
        dgs = dgs[::-1]+get_delta_g(st5,st3,dg5,dg3)[:15]
        delgaj = delgaj.append(pd.Series(dict(zip(cols,[m,dist]+dgs))),ignore_index=True)
delgbj = delgbj.fillna(0)
delgaj = delgaj.fillna(0)

In [69]:
coeffsbj = pd.DataFrame(columns=range(-WIN,WIN))
coeffsaj = pd.DataFrame(columns=range(-WIN,WIN))
for l1 in range(1,1+WIN):
    for l2 in range(1,1+WIN):
        coeffbj = [0]*(WIN-l1)+[1]*l1+[-1]*l2+[0]*(WIN-l2)
        coeffaj = [0]*(WIN-l1)+[-1]*l1+[1]*l2+[0]*(WIN-l2)
        labelbj = 'L1=%s, L2=%s'%(l1,l2)
        labelaj = 'L3=%s, L4=%s'%(l1,l2)
        coeffsbj.loc[labelbj] = pd.Series(dict(zip(range(-WIN,WIN),coeffbj)))
        coeffsaj.loc[labelaj] = pd.Series(dict(zip(range(-WIN,WIN),coeffaj)))

In [70]:
delgbj = delgbj.join(delgbj[range(-WIN,WIN)].dot(coeffsbj.T))
delgaj = delgaj.join(delgaj[range(-WIN,WIN)].dot(coeffsaj.T))

In [71]:
L1, L2, L3, L4 = 3,3,5,5

In [72]:
jcpostbl = pd.DataFrame()
for m in folded:
    ps, pe = get_pre_position(m)
    symstr = get_sym_str(m)
    for x5,x3 in list_all_junctions(m,'basal'):
        dist = -count_len(symstr[x5-1:ps-1])
        jcpostbl.loc[m,dist+int(x5>=ps)] = '%s,%s'%(x5,x3)

    for y5,y3 in list_all_junctions(m,'apical'):
        if y5>ps: 
            dist = count_len(symstr[ps-1:y5-1])
            jcpostbl.loc[m,dist+1] = '%s,%s'%(y5,y3)
jcpostbl = jcpostbl[sorted(jcpostbl.columns)]

bjtbl = pd.pivot_table(delgbj, index=['mir'], columns=['dist'],
                       values='L1=%s, L2=%s'%(L1,L2), aggfunc=np.sum)
ajtbl = pd.pivot_table(delgaj, index=['mir'], columns=['dist'],
                       values='L3=%s, L4=%s'%(L3,L4), aggfunc=np.sum)

In [73]:
LS = range(-15,-10)
US = range(19,26)
TS = range(31,39)
flag = 0
jcpairs = {}
while len(jcpairs)<len(folded):
    ms = [m for m in folded if m not in jcpairs]
    for m in ms:
        bjs = delgbj[delgbj['mir']==m].set_index('dist')['L1=%s, L2=%s'%(L1,L2)].to_dict()
        ajs = delgaj[delgaj['mir']==m].set_index('dist')['L3=%s, L4=%s'%(L3,L4)].to_dict()
        pairs = [(b,a) for b in bjs for a in ajs if a-b in TS and b in LS and a in US]     
        if pairs:
            jcpairs[m] = max(pairs,key=lambda (x,y):bjs[x]+ajs[y])
    if flag%3==1:
        TS = range(min(TS)-1,max(TS)+2); flag+=1
    elif flag%3==2:
        US = range(min(US)-1,max(US)+2); flag+=1
    else:
        LS = range(min(LS)-1,max(LS)+2); flag+=1

In [74]:
jcpostbl.loc['hsa-mir-656',-14] = '19,107'
jcpairs['hsa-mir-18a'] = (-13,22)
jcpairs['hsa-mir-18b'] = (-8,20)
jcpairs['hsa-mir-19b-2'] = (-13,21)
jcpairs['hsa-mir-20b'] = (-12,19)
jcpairs['hsa-mir-24-2'] = (-13,22)
jcpairs['hsa-mir-26a-1'] = (-14,22)
jcpairs['hsa-mir-27b'] = (-13,21)
jcpairs['hsa-mir-29a'] = (-14,23)
jcpairs['hsa-mir-32'] = (-14,22)
jcpairs['hsa-mir-93'] = (-13,21)
jcpairs['hsa-mir-105-2'] = (-13,22)
jcpairs['hsa-mir-106a'] = (-13,22)
jcpairs['hsa-mir-122'] = (-13,22)
jcpairs['hsa-mir-126'] = (-7,19)
jcpairs['hsa-mir-129-1'] = (-13,23)
jcpairs['hsa-mir-130b'] = (-12,19)
jcpairs['hsa-mir-134'] = (-10,19)
jcpairs['hsa-mir-136'] = (-16,22)
jcpairs['hsa-mir-143'] = (-11,21)
jcpairs['hsa-mir-144'] = (-13,19)
jcpairs['hsa-mir-146b'] = (-10,19)
jcpairs['hsa-mir-148a'] = (-13,20)
jcpairs['hsa-mir-151a'] = (-9,21)
jcpairs['hsa-mir-153-1'] = (-13,22)
jcpairs['hsa-mir-190b'] = (-10,21)
jcpairs['hsa-mir-195'] = (-9,22)
jcpairs['hsa-mir-199a-1'] = (-10,22)
jcpairs['hsa-mir-199a-2'] = (-14,23)
jcpairs['hsa-mir-200b'] = (-12,21)
jcpairs['hsa-mir-200c'] = (-13,23)
jcpairs['hsa-mir-202'] = (-15,24)
jcpairs['hsa-mir-218-2'] = (-8,21)
jcpairs['hsa-mir-221'] = (-14,22)
jcpairs['hsa-mir-302c'] = (-2,21)
jcpairs['hsa-mir-302d'] = (-11,20)
jcpairs['hsa-mir-325'] = (-14,19)
jcpairs['hsa-mir-335'] = (-10,26)
jcpairs['hsa-mir-337'] = (-15,22)
jcpairs['hsa-mir-374b'] = (-10,23)
jcpairs['hsa-mir-372'] = (-12,21)
jcpairs['hsa-mir-376a-2'] = (-15,21)
jcpairs['hsa-mir-376b'] = (-15,21)
jcpairs['hsa-mir-377'] = (-14,22)
jcpairs['hsa-mir-379'] = (-14,21)
jcpairs['hsa-mir-410'] = (-12,21)
jcpairs['hsa-mir-433'] = (-12,21)
jcpairs['hsa-mir-452'] = (-13,23)
jcpairs['hsa-mir-486-1'] = (-14,21)
jcpairs['hsa-mir-503'] = (-13,22)
jcpairs['hsa-mir-513a-1'] = (-13,24)
jcpairs['hsa-mir-518a-1'] = (-13,21)
jcpairs['hsa-mir-518a-2'] = (-14,21)
jcpairs['hsa-mir-518b'] = (-13,22)
jcpairs['hsa-mir-518d'] = (-9,23)
jcpairs['hsa-mir-519d'] = (-14,25)
jcpairs['hsa-mir-520a'] = (-14,21)
jcpairs['hsa-mir-520f'] = (-14,19)
jcpairs['hsa-mir-525'] = (-14,21)
jcpairs['hsa-mir-526a-1'] = (-6,21)
jcpairs['hsa-mir-526b'] = (-13,21)
jcpairs['hsa-mir-541'] = (-15,24)
jcpairs['hsa-mir-589'] = (-13,26)
jcpairs['hsa-mir-641'] = (-14,24)
jcpairs['hsa-mir-656'] = (-14,21)
jcpairs['hsa-mir-708'] = (-13,21)
jcpairs['hsa-mir-758'] = (-11,22)
jcpairs['hsa-mir-766'] = (-12,21)
jcpairs['hsa-mir-770'] = (-11,22)
jcpairs['hsa-mir-1185-1'] = (-14,21)
jcpairs['hsa-mir-1185-2'] = (-14,21)
jcpairs['hsa-mir-1245b'] = (-17,19)
jcpairs['hsa-mir-1287'] = (-13,20)
jcpairs['hsa-mir-1908'] = (-14,18)
jcpairs['hsa-mir-2278'] = (-8,22)
jcpairs['hsa-mir-2355'] = (-12,26)
jcpairs['hsa-mir-2467'] = (-15,22)
jcpairs['hsa-mir-3173'] = (-5,24)
jcpairs['hsa-mir-3186'] = (-10,22)
jcpairs['hsa-mir-3612'] = (-9,24)
jcpairs['hsa-mir-3681'] = (-9,23)
jcpairs['hsa-mir-3918'] = (-16,23)
jcpairs['hsa-mir-3944'] = (-13,22)
jcpairs['hsa-mir-4664'] = (-6,20)
jcpairs['hsa-mir-4677'] = (-15,22)
jcpairs['hsa-mir-510'] = (-16,22)

---
### 5. Get features

In [76]:
kims3file = 'publication/Kim2021_TableS3.xlsx'
kims3 = pd.ExcelFile(kims3file).parse('s3-2. IVP results',header=3,index_col=0)
effs = dict(kims3['Cleavage Efficiency'])
homs = dict(kims3['Cleavage Homogeneity'])
kims3.head(1)

,Cleavage Efficiency,Cleavage Homogeneity,5' miRBase site,3' miRBase site,5' cleavage site,3' cleavage site,5' alternative site,3' alternative site,Fraction of alternative site,5p-nick processing ratio,3p-nick processing ratio,Inverse processing ratio
Pri-miRNA,,,,,,,,,,,,
hsa-let-7a-1,4.051311,0.882549,25,96,25.0,96.0,24.0,97.0,0.017713,0.004202,0.010504,0.008403


In [139]:
kims6file = 'publication/Kim2021_TableS6.xlsx'
kims6 = pd.ExcelFile(kims6file).parse('s6-2. IVP results',header=3,index_col=0)
effssr = dict(kims6['Cleavage Efficiency'])
homssr = dict(kims6['Cleavage Homogeneity'])
kims6.head(1)

,Cleavage Efficiency,Cleavage Homogeneity,5' miRBase site,3' miRBase site,5' cleavage site,3' cleavage site,5' alternative site,3' alternative site,Fraction of alternative site,5p-nick processing ratio,3p-nick processing ratio,Inverse processing ratio
Pri-miRNA,,,,,,,,,,,,
hsa-let-7a-1,6.380507,0.870736,25,96,25.0,96.0,NaN,NaN,NaN,0.001255,0.027604,0.007528


In [77]:
print '>> TS'
bins = [(20,33),(34,34),(35,35),(36,36),(37,50)]
msl = [[m for m in folded if lb<=jcpairs[m][1]-jcpairs[m][0]<=ub] for lb,ub in bins]
pes = [[effs[m] for m in ms] for ms in msl]
phs = [[homs[m] for m in ms] for ms in msl]
labels1 = [33,34,35,36,37]
for lab,pe,ph in zip(labels1,pes,phs):
    print '%s\t%s\t%.4f\t%.4f'%(lab,len(pe),mannwhitneyu(pes[2],pe)[1]*2,
                             mannwhitneyu(phs[2],ph)[1]*2)

print '>> US'
bins = [(-5,20),(21,21),(22,22),(23,23),(24,40)]
msl = [[m for m in folded if lb<=jcpairs[m][1]<=ub] for lb,ub in bins]
pes = [[effs[m] for m in ms] for ms in msl]
phs = [[homs[m] for m in ms] for ms in msl]
labels1 = [20,21,22,23,24]
for lab,pe,ph in zip(labels1,pes,phs):
    print '%s\t%s\t%.4f\t%.4f'%(lab,len(pe),mannwhitneyu(pes[2],pe)[1]*2,
                             mannwhitneyu(phs[2],ph)[1]*2)

print '>> LS'
bins = [(-5,11),(12,12),(13,13),(14,14),(15,30)]
msl = [[m for m in folded if lb<=-jcpairs[m][0]<=ub] for lb,ub in bins]
pes = [[effs[m] for m in ms] for ms in msl]
phs = [[homs[m] for m in ms] for ms in msl]
labels1 = [11,12,13,14,15]
for lab,pe,ph in zip(labels1,pes,phs):
    print '%s\t%s\t%.4f\t%.4f'%(lab,len(pe),mannwhitneyu(pes[2],pe)[1]*2,
                             mannwhitneyu(phs[2],ph)[1]*2)

>> TS
33	121	0.0109	0.0014
34	76	0.5760	0.9187
35	105	0.9991	0.9991
36	89	0.3058	0.2578
37	85	0.0504	0.0043
>> US
20	54	0.2762	0.3936
21	120	0.5545	0.9488
22	155	0.9995	0.9995
23	85	0.2277	0.2159
24	62	0.0672	0.1034
>> LS
11	83	0.0001	0.0000
12	86	0.0090	0.0090
13	157	0.9995	0.9995
14	97	0.6885	0.0339
15	53	0.0006	0.0049


In [102]:
LENGTH = 125
BJWIN = 8
shannon_ens = defaultdict(list)
for m in folded:
    shens = s3shan.loc[m]
    bj,aj = jcpairs[m]
    bj5,bj3 = map(int,jcpostbl.loc[m,bj].split(','))
    aj5,aj3 = map(int,jcpostbl.loc[m,aj].split(','))    
    ctinfo = ctinfos[m]
    shannon_ens['stem'].extend([ shens[p] for p in range(bj5,aj5+1)+range(aj3,bj3+1) if ctinfo[p]!=0 ])
    shannon_ens['loop'].extend([ shens[p] for p in range(aj5+1,aj3) if ctinfo[p]!=0 ])
    shannon_ens['basal'].extend([ shens[p] for p in range(max(1,bj5-BJWIN),bj5)+range(bj3+1,min(125,bj3+1+BJWIN))
                                  if ctinfo[p]!=0 ])

In [106]:
xs = np.arange(0,.61,.02)
bins = np.arange(0,.63,.02)-.01
ys1 = np.histogram(shannon_ens['stem'],bins=bins)[0]
ys2 = np.histogram(shannon_ens['loop'],bins=bins)[0]
ys3 = np.histogram(shannon_ens['basal'],bins=bins)[0]
cut = np.percentile(shannon_ens['stem'],95)
frac_loop = len([se for se in shannon_ens['loop'] if se>=cut])/len(shannon_ens['loop'])
frac_basal = len([se for se in shannon_ens['basal'] if se>=cut])/len(shannon_ens['basal'])
print round(cut,3), round(frac_loop,3), round(frac_basal,3)

0.2 0.061 0.164


In [107]:
bs5s, bs3s = {}, {}
bes = {}
for m in folded:
    shens = s3shan.loc[m]
    bj,aj = jcpairs[m]
    bj5,bj3 = map(int,jcpostbl.loc[m,bj].split(','))
    aj5,aj3 = map(int,jcpostbl.loc[m,aj].split(','))    
    ctinfo = ctinfos[m]
    
    outpairs = [(i,p) for i,p in ctinfo if p>0 and not (i in range(bj5,bj3+1) or p in range(bj5,bj3+1))]
    defpairs = [(i,p) for i,p in outpairs if shens[i]<cut and shens[p]<cut]
    be5 = max([i for i,p in defpairs if i<bj5]+[0])
    be3 = min([i for i,p in defpairs if i>bj3]+[126])
    bes[m] = (be5,be3)
    bs5s[m] = bj5-be5-1
    bs3s[m] = be3-bj3-1

In [113]:
mismatch = {}
bulged5, bulged3 = {}, {}
mmtbl = pd.DataFrame(index=folded,columns=range(-15,25))
bg5tbl = pd.DataFrame(index=folded,columns=range(-15,25))
bg3tbl = pd.DataFrame(index=folded,columns=range(-15,25))
for m in folded:
    ps, pe = get_pre_position(m)
    symstr = get_sym_str(m)
    bj,aj = jcpairs[m]
    bj5,bj3 = map(int,jcpostbl.loc[m,bj].split(','))
    aj5,aj3 = map(int,jcpostbl.loc[m,aj].split(','))
    stem5sts = symstr[bj5-1:aj5]
    mismatch[m] = stem5sts.count('S')
    bulged5[m] = symstr[bj5-1:aj5].count('A')
    bulged3[m] = symstr[aj3-1:bj3].count('A')
    
    usstr = symstr[ps-1:aj5].replace('A','')+symstr[aj5:aj5+2]
    lsstr = symstr[bj5-3:bj5-1]+symstr[bj5-1:ps-1].replace('A','')
    usstr = [0 if s=='M' else 1 for s in usstr]
    lsstr = [0 if s=='M' else 1 for s in lsstr]
    strd = dict(zip(range(25),usstr)); strd.update(dict(zip(range(-1,-16,-1),lsstr[::-1])))
    mmtbl.loc[m] = pd.Series(strd)
    
    lstem = abs(bj)
    str5 = re.sub('[A]+','A',symstr[bj5-1:aj5])
    str3 = re.sub('[A]+','A',symstr[aj3-1:bj3][::-1])
    bgd5 = {r.start()-lstem-str5[:r.start()].count('A'):1 for r in re.finditer('MAM',str5)}
    bgd3 = {r.start()-lstem-str3[:r.start()].count('A'):1 for r in re.finditer('MAM',str3)}
    bg5tbl.loc[m] = pd.Series(bgd5)
    bg3tbl.loc[m] = pd.Series(bgd3)
bg5tbl = bg5tbl.fillna(0).astype(int)
bg3tbl = bg3tbl.fillna(0).astype(int)

In [128]:
bulgepos = -11
bg3ms = bg3tbl[bg3tbl[bulgepos]==1].index
bg3nuc, bg3seq = {}, {}
for m in bg3ms:
    ps, pe = get_pre_position(m)
    symstr = get_sym_str(m)
    bj,aj = jcpairs[m]
    lstem = abs(bj)
    ustem = abs(aj)
    bj5,bj3 = map(int,jcpostbl.loc[m,bj].split(','))
    aj5,aj3 = map(int,jcpostbl.loc[m,aj].split(','))
    str5 = symstr[bj5-1:aj5]
    seq5 = constseqs[m][bj5-1:aj5].replace('T','U')    
    str3 = symstr[aj3-1:bj3]
    seq3 = constseqs[m][aj3-1:bj3].replace('T','U')
    for r in re.finditer('M[A]+M',str3):
        if count_len(str3[:r.start()+1])==ustem+abs(bulgepos)-1:
            s3,e3 = r.span()
            break
    else:
        continue
    for i in range(len(str5)):
        if str5[i-1:i+1]=='MM' and count_len(str5[i:])==ustem+abs(bulgepos)-1:
            s5,e5 = i-1,i+1
            break
    else:
        continue
    bg3seq[m] = seq3[s3+1:e3-1]
    bg3nuc[m] = map(lambda (x,y):x+y,
                    zip(seq5[s5-1:e5+1], seq3[e3-1:e3+1][::-1]+seq3[s3-1:s3+1][::-1]))
bg3nuc['hsa-mir-2115'] = ('CG','CG','CG','AU')
bg3seq['hsa-mir-2115'] = 'A'
print len(bg3ms), len(bg3nuc)

28 28


In [133]:
bgwgs = [m for m in bg3ms if bg3nuc[m][1] in 'CG UG' and bg3seq[m] in 'U A' and bg3nuc[m][2] in 'CG UG']
print len(bgwgs)

9


In [143]:
tbl = pd.DataFrame(index=folded, columns=range(1,126))
for m in folded:
    tbl.loc[m] = pd.Series(dict(ctinfos[m]))
tbl.head(1)

,1,2,3,4,5,6,7,8,9,10,...,116,117,118,119,120,121,122,123,124,125
hsa-let-7a-1,124,123,122,121,120,119,118,117,0,0,...,0,8,7,6,5,4,3,2,1,0


In [144]:
tbl.to_csv('publication/TableS4_1__SHAPE-based_pri-miRNA_structures.csv')

In [137]:
COLS = ['Library','Fold param.','i th str.',
        'Apical junction position','Basal junction position','Basal segment boundaries',
        'Stem length (bp)','Lower stem length (bp)','Upper stem length (bp)','Mismatches','5p bulges','3p bulges',
        'Mismatch at -6','bGWG at -10/-11',
        'Apical loop size (nt)',"5' basal segment length (nt)","3' basal segment length (nt)",
        'Cleavage efficiency (MP)','Cleavage homogeneity (MP)',
        'Cleavage efficiency (MP+SRSF3)','Cleavage Homogeneity (MP+SRSF3)']
PARAM_DICT = {'m18':'-m 1.8 -i -0.6',
              'm24':'-m 2.4 -i -0.8',
              'm12':'-m 1.2 -i -0.4'}

In [140]:
tbl = pd.DataFrame(index=folded, columns=COLS)
for m in folded:
    lib,param = ctfileinfos[m][0].split('/')[-1].split('.')[0].split('_')[1:]
    strnum = ctfileinfos[m][1]
    tbl.loc[m,'Library'] = lib
    tbl.loc[m,'Fold param.'] = PARAM_DICT[param]
    tbl.loc[m,'i th str.'] = strnum
       
    bj,aj = jcpairs[m]
    bj5,bj3 = map(int,jcpostbl.loc[m,bj].split(','))
    aj5,aj3 = map(int,jcpostbl.loc[m,aj].split(','))
    be5,be3 = bes[m]
    tbl.loc[m,'Apical junction position'] = '%i, %i'%(aj5,aj3)
    tbl.loc[m,'Basal junction position'] = '%i, %i'%(bj5,bj3)
    tbl.loc[m,'Basal segment boundaries'] = '%i, %i'%(be5,be3)
    
    tbl.loc[m,'Stem length (bp)'] = aj-bj
    tbl.loc[m,'Lower stem length (bp)'] = -bj
    tbl.loc[m,'Upper stem length (bp)'] = aj
    tbl.loc[m,'Mismatches'] = mismatch[m]
    tbl.loc[m,'5p bulges'] = bulged5[m]
    tbl.loc[m,'3p bulges'] = bulged3[m]
    
    tbl.loc[m,'Mismatch at -6'] = mmtbl.loc[m,-6]
    tbl.loc[m,'bGWG at -10/-11'] = int(m in bgwgs)
    
    tbl.loc[m,'Apical loop size (nt)'] = aj3-aj5-1
    tbl.loc[m,"5' basal segment length (nt)"] = bs5s[m]
    tbl.loc[m,"3' basal segment length (nt)"] = bs3s[m]
    
    tbl.loc[m,'Cleavage efficiency (MP)'] = effs[m]/effs['hsa-mir-30a']
    tbl.loc[m,'Cleavage homogeneity (MP)'] = homs[m]/homs['hsa-mir-30a'] 
    tbl.loc[m,'Cleavage efficiency (MP+SRSF3)'] = effssr[m]/effs['hsa-mir-30a']
    tbl.loc[m,'Cleavage Homogeneity (MP+SRSF3)'] = homssr[m]/homs['hsa-mir-30a'] 
tbl.head(1)

,Library,Fold param.,i th str.,Apical junction position,Basal junction position,Basal segment boundaries,Stem length (bp),Lower stem length (bp),Upper stem length (bp),Mismatches,...,3p bulges,Mismatch at -6,bGWG at -10/-11,Apical loop size (nt),5' basal segment length (nt),3' basal segment length (nt),Cleavage efficiency (MP),Cleavage homogeneity (MP),Cleavage efficiency (MP+SRSF3),Cleavage Homogeneity (MP+SRSF3)
hsa-let-7a-1,rep2-1,-m 2.4 -i -0.8,1,"46, 74","11, 109","8, 117",34,13,21,0,...,2,0,0,27,2,7,1.41184,1.12131,2.22354,1.1063


In [141]:
tbl.to_csv('publication/TableS4__Structural_features_of_pri-miRNAs.csv')

---

In [162]:
fincts = [ f for f in listdir(FIG_DIR) if f.endswith('.ct') ]
for m in folded:
    ctfile,i,rep = ctfileinfos[m]
    finct = ctfile.replace('.ct','_%i.ct'%i).replace('structures','figures')
    if finct.split('/')[-1] in fincts:
        newfile = 'publication/structures/SHAPE/%s.ct'%m
        !cp $finct $newfile
    else:
        print m

In [148]:
ctfile = 'shape/assem/structures/IRES-domainII_rep2-2_1.ct'
newfile = 'publication/structures/SHAPE/IRES-domainII.ct'
!cp $ctfile $newfile

ctfile = 'shape/assem/structures/U1-snRNA_rep2-2_1.ct'
newfile = 'publication/structures/SHAPE/U1-snRNA.ct'
!cp $ctfile $newfile

ctfile = 'shape/assem/structures/Yeast-tRNAasp_rep2-2_2.ct'
newfile = 'publication/structures/SHAPE/Yeast-tRNAasp.ct'
!cp $ctfile $newfile